<a href="https://colab.research.google.com/github/eli-osherovich/eol/blob/master/mnist_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist, fashion_mnist

In [2]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-c79ae2d4-448b-8432-666f-b372ae8c8e4d)


In [3]:
tf.config.optimizer.set_jit(True)

In [4]:
# the data, split between train and test sets
dataset = fashion_mnist
dataset = mnist


(x_train, y_train), (x_test, y_test) = dataset.load_data()

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train = x_train.astype(np.float32)/255
x_test = x_test.astype(np.float32)/255
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


In [5]:
def gen_model(activation="relu"):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(28, 28, 1)))

  model.add(tf.keras.layers.Conv2D(128, 5, activation=activation, padding="same"))
  model.add(tf.keras.layers.AveragePooling2D(2))
  
  model.add(tf.keras.layers.Conv2D(128, 5, activation=activation))
  model.add(tf.keras.layers.AveragePooling2D())

  model.add(tf.keras.layers.Conv2D(128, 5, activation=activation))
  
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(128, activation=activation))
  model.add(tf.keras.layers.Dense(10))
  return model

def compile_model(model, lr=0.001):
  model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=keras.optimizers.Adam(learning_rate=lr),
                metrics=['accuracy'])
  

early_stop = tf.keras.callbacks.EarlyStopping(
      monitor='val_accuracy',
      patience=20,
      restore_best_weights=True,
)

In [6]:
%%time
model = gen_model()
compile_model(model)
model.fit(x_train, y_train,
          batch_size=128,
          epochs=1000,
          verbose=1,
          callbacks=[early_stop],
          validation_data=(x_test, y_test),
          validation_batch_size=2048)


Epoch 1/1000
469/469 [==============================] - 4s 9ms/step - loss: 0.1848 - accuracy: 0.9436 - val_loss: 0.0532 - val_accuracy: 0.9840
Epoch 2/1000
469/469 [==============================] - 4s 10ms/step - loss: 0.0515 - accuracy: 0.9838 - val_loss: 0.0459 - val_accuracy: 0.9847
Epoch 3/1000
469/469 [==============================] - 2s 5ms/step - loss: 0.0343 - accuracy: 0.9890 - val_loss: 0.0342 - val_accuracy: 0.9882
Epoch 4/1000
469/469 [==============================] - 3s 5ms/step - loss: 0.0267 - accuracy: 0.9913 - val_loss: 0.0239 - val_accuracy: 0.9918
Epoch 5/1000
469/469 [==============================] - 2s 5ms/step - loss: 0.0208 - accuracy: 0.9933 - val_loss: 0.0276 - val_accuracy: 0.9915
Epoch 6/1000
469/469 [==============================] - 2s 5ms/step - loss: 0.0181 - accuracy: 0.9942 - val_loss: 0.0316 - val_accuracy: 0.9905
Epoch 7/1000
469/469 [==============================] - 2s 5ms/step - loss: 0.0155 - accuracy: 0.9950 - val_loss: 0.0259 - val_accuracy

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.04971187561750412
Test accuracy: 0.9947000741958618
